In [2]:
# --- Paths (ajusta si es necesario) ---
PATH_ALERTS = "../data/high_alerts_enriched_with_tx.csv"

import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# --- Carga ---
alerts = pd.read_csv(PATH_ALERTS, encoding="utf-8-sig")
alerts.columns = [str(c).strip().replace("\ufeff","") for c in alerts.columns]

# --- Normalizaciones básicas ---
alerts["rule_code"] = alerts["rule_code"].astype(str).str.strip()
alerts["status"]    = alerts["status"].astype(str).str.strip()
alerts["customer_sub_type"] = alerts["customer_sub_type"].astype(str).str.strip()
alerts["created_at"] = pd.to_datetime(alerts["created_at"], errors="coerce")
alerts["created_date"] = alerts["created_at"].dt.date

# subject_ids: mantén el primero (suele ser 1-to-1 con el cliente en este reporte)
alerts["subject_id"] = alerts["subject_ids"].astype(str).str.split(",").str[0].str.strip()

# external_transaction_ids: lista de TX; las “NA” las tratamos como vacías
def split_ids(s):
    if pd.isna(s): 
        return []
    parts = [p.strip() for p in str(s).split(",")]
    return [p for p in parts if p and p.upper() != "NA"]

alerts["tx_list"] = alerts["external_transaction_ids"].apply(split_ids)

print("Filas:", len(alerts), "| Reglas distintas:", alerts["rule_code"].nunique(), "| Segmentos:", alerts["customer_sub_type"].nunique())
alerts.head(3)


Filas: 2147 | Reglas distintas: 35 | Segmentos: 9


,alert_id,rule_code,subject_ids,subject_names,number_of_transactions,created_at,status,external_transaction_ids,tx_direction,tx_base_amount,customer_type,customer_account_balance,customer_networth,customer_income,customer_expected_amount,customer_sub_type,created_date,subject_id,tx_list
0,82271,AAD-LA,26375,MOREL BULICIC JORGE RAFAEL,2,2025-03-07 12:27:52.142219+00:00,Not Suspicious,"201955096, 68816944","NA, NA","NA, NA","NA, NA","NA, NA","NA, NA","NA, NA","NA, NA",SIN_SEGMENTO,2025-03-07,26375,"[201955096, 68816944]"
1,83230,P-TLO,24618,GARCIA LABORA WALDO,1,2025-03-11 12:33:06.388175+00:00,Not Suspicious,202249097,Outbound,230017346.0,Individual,366027051.0,SIN CLASIFICACION,Entre 5 y 10 millones,1500000000.0,Retail,2025-03-11,24618,[202249097]
2,83231,PGAV-OUT,24618,GARCIA LABORA WALDO,1,2025-03-11 12:33:06.388175+00:00,Not Suspicious,202249097,Outbound,230017346.0,Individual,366027051.0,SIN CLASIFICACION,Entre 5 y 10 millones,1500000000.0,Retail,2025-03-11,24618,[202249097]


In [3]:
# --- (Opcional) Mapa simple de 'rule_code' → grupo, ajusta si quieres más fino ---
GROUP_MAP = {
    # Monto transaccional
    "IN>AVG": "Monto transaccional", "OUT>AVG": "Monto transaccional",
    "P-TLI": "Monto transaccional", "P-TLO": "Monto transaccional",
    "P-LVAL": "Monto transaccional", "P-LBAL": "Monto transaccional",
    "P-HSUMO": "Monto transaccional", "P-HSUMI": "Monto transaccional",
    "PGAV-IN": "Monto transaccional", "PGAV-OUT": "Monto transaccional",
    # Frecuencia (ej.)
    "HANUMI": "Frecuencia", "HANUMO": "Frecuencia", "HNR-IN":"Frecuencia", "HNR-OUT":"Frecuencia", 
    # Contraparte (ej.)
    "NCU":"Contraparte", "NCOU":"Contraparte","NBCOU":"Contraparte",
    # Otros
    "IN-OUT-1": "Comportamiento transaccional", "OUT>%IN":"Comportamiento transaccional", "IN>%OUT":"Comportamiento transaccional",
    "SEC":"Comportamiento transaccional", "DORMANT":"Otros (descriptivos)",
}

alerts["group"] = alerts["rule_code"].map(GROUP_MAP).fillna("No clasificado")

# --- KPIs por regla (global) ---
def _safe_div(a,b):
    return (a/b*100) if b else 0.0

by_rule = (alerts
           .groupby("rule_code", dropna=False)
           .agg(total_alerts=("alert_id","count"),
                suspicious=("status", lambda s: (s=="Suspicious").sum()),
                not_suspicious=("status", lambda s: (s=="Not Suspicious").sum()))
           .reset_index())
by_rule["reviewed"] = by_rule["suspicious"] + by_rule["not_suspicious"]
by_rule["fp_rate%"] = by_rule.apply(lambda r: _safe_div(r["not_suspicious"], r["reviewed"]), axis=1)
by_rule["precision%"] = by_rule.apply(lambda r: _safe_div(r["suspicious"], r["reviewed"]), axis=1)

by_rule = by_rule.sort_values(["reviewed","total_alerts"], ascending=False).reset_index(drop=True)
display(by_rule.head(30))

# --- KPIs por grupo y regla (ordenado por FP rate) ---
by_group_rule = (alerts
    .groupby(["group","rule_code"], dropna=False)
    .agg(total_alerts=("alert_id","count"),
         suspicious=("status", lambda s: (s=="Suspicious").sum()),
         not_suspicious=("status", lambda s: (s=="Not Suspicious").sum()))
    .reset_index())
by_group_rule["reviewed"] = by_group_rule["suspicious"] + by_group_rule["not_suspicious"]
by_group_rule["fp_rate%"] = by_group_rule.apply(lambda r: _safe_div(r["not_suspicious"], r["reviewed"]), axis=1)
by_group_rule["precision%"] = by_group_rule.apply(lambda r: _safe_div(r["suspicious"], r["reviewed"]), axis=1)

by_group_rule = by_group_rule.sort_values(["group","fp_rate%","total_alerts"], ascending=[True,False,False]).reset_index(drop=True)
display(by_group_rule)


,rule_code,total_alerts,suspicious,not_suspicious,reviewed,fp_rate%,precision%
0,PGAV-OUT,331,11,316,327,96.64,3.36
1,OUT>%IN,214,6,207,213,97.18,2.82
2,HNR-IN,187,16,170,186,91.40,8.60
3,HASUMI,135,7,126,133,94.74,5.26
4,PGAV-IN,99,9,87,96,90.62,9.38
5,P-LVAL,88,7,79,86,91.86,8.14
6,P-TLI,85,4,80,84,95.24,4.76
7,OCMC_1,84,8,73,81,90.12,9.88
8,HASUMO,76,3,72,75,96.00,4.00
9,IN>%OUT,74,4,70,74,94.59,5.41


,group,rule_code,total_alerts,suspicious,not_suspicious,reviewed,fp_rate%,precision%
0,Comportamiento transaccional,SEC,19,0,19,19,100.00,0.00
1,Comportamiento transaccional,OUT>%IN,214,6,207,213,97.18,2.82
2,Comportamiento transaccional,IN>%OUT,74,4,70,74,94.59,5.41
3,Comportamiento transaccional,IN-OUT-1,17,1,15,16,93.75,6.25
4,Contraparte,NCU,28,2,26,28,92.86,7.14
5,Frecuencia,HANUMO,23,1,21,22,95.45,4.55
6,Frecuencia,HNR-IN,187,16,170,186,91.40,8.60
7,Frecuencia,HANUMI,59,7,52,59,88.14,11.86
8,Frecuencia,HNR-OUT,50,9,41,50,82.00,18.00
9,Monto transaccional,P-HSUMI,72,1,71,72,98.61,1.39


In [4]:
# Explota 'tx_list' → una fila por (alert_id, rule_code, subject_id, tx_id)
expl = alerts.explode("tx_list").rename(columns={"tx_list":"tx_id"}).copy()

# Dos llaves para medir co-disparo:
# 1) (subject_id, tx_id)  → co-alertas sobre la MISMA transacción del MISMO cliente
# 2) (subject_id, created_date) → co-alertas el MISMO día para el MISMO cliente (por si falta el tx_id)
expl["has_tx"] = expl["tx_id"].notna()

print("Con tx_id:", int(expl["has_tx"].sum()), "| Sin tx_id:", int((~expl["has_tx"]).sum()))
expl.head(3)


Con tx_id: 2774 | Sin tx_id: 0


,alert_id,rule_code,subject_ids,subject_names,number_of_transactions,created_at,status,external_transaction_ids,tx_direction,tx_base_amount,...,customer_account_balance,customer_networth,customer_income,customer_expected_amount,customer_sub_type,created_date,subject_id,tx_id,group,has_tx
0,82271,AAD-LA,26375,MOREL BULICIC JORGE RAFAEL,2,2025-03-07 12:27:52.142219+00:00,Not Suspicious,"201955096, 68816944","NA, NA","NA, NA",...,"NA, NA","NA, NA","NA, NA","NA, NA",SIN_SEGMENTO,2025-03-07,26375,201955096,No clasificado,True
0,82271,AAD-LA,26375,MOREL BULICIC JORGE RAFAEL,2,2025-03-07 12:27:52.142219+00:00,Not Suspicious,"201955096, 68816944","NA, NA","NA, NA",...,"NA, NA","NA, NA","NA, NA","NA, NA",SIN_SEGMENTO,2025-03-07,26375,68816944,No clasificado,True
1,83230,P-TLO,24618,GARCIA LABORA WALDO,1,2025-03-11 12:33:06.388175+00:00,Not Suspicious,202249097,Outbound,230017346.0,...,366027051.0,SIN CLASIFICACION,Entre 5 y 10 millones,1500000000.0,Retail,2025-03-11,24618,202249097,Monto transaccional,True


In [5]:
from itertools import combinations
from collections import Counter, defaultdict

def dice_similarity(A, B):
    # Dice = 2|A∩B| / (|A|+|B|)
    if not A or not B: 
        return 0.0
    inter = len(A & B)
    return 2*inter / (len(A)+len(B))

def co_stats(df, key_cols):
    # Construye conjuntos por regla en torno a una "unidad" (A = set de unidades alertadas por la regla)
    units_by_rule = df.groupby("rule_code")[key_cols].apply(lambda x: set(map(tuple, x.drop_duplicates().to_records(index=False)))).to_dict()
    sizes = {r: len(s) for r,s in units_by_rule.items()}

    # Top pares por Dice
    rows = []
    rules = sorted(units_by_rule.keys())
    for a,b in combinations(rules, 2):
        s = dice_similarity(units_by_rule[a], units_by_rule[b])
        if s > 0:
            rows.append((a,b,s, sizes[a], sizes[b]))
    pair_df = pd.DataFrame(rows, columns=["rule_a","rule_b","dice","|A|","|B|"]).sort_values("dice", ascending=False).head(50)

    # Subconjuntos: A ⊆ B cuando |A∩B|=|A|
    subset_rows = []
    for a,b in combinations(rules, 2):
        A, B = units_by_rule[a], units_by_rule[b]
        if A and A.issubset(B):
            subset_rows.append((a,b, len(A), len(B), "A ⊆ B"))
        if B and B.issubset(A):
            subset_rows.append((b,a, len(B), len(A), "B ⊆ A"))
    subset_df = pd.DataFrame(subset_rows, columns=["subset_rule","superset_rule","|subset|","|superset|","flag"]).drop_duplicates()

    return pair_df, subset_df, sizes

# 4.1) Misma transacción (cliente, tx)
with_tx = expl[expl["has_tx"]].copy()
pair_tx, subset_tx, sizes_tx = co_stats(with_tx, ["subject_id","tx_id"])
print("=== Similaridad (Dice) por MISMA transacción (cliente, tx) ===")
display(pair_tx)
print("=== Subconjuntos por MISMA transacción (cliente, tx) ===")
display(subset_tx.head(30))

# 4.2) Mismo día (cliente, fecha)
pair_day, subset_day, sizes_day = co_stats(expl, ["subject_id","created_date"])
print("=== Similaridad (Dice) por MISMO DÍA (cliente, fecha) ===")
display(pair_day)
print("=== Subconjuntos por MISMO DÍA (cliente, fecha) ===")
display(subset_day.head(30))


=== Similaridad (Dice) por MISMA transacción (cliente, tx) ===


,rule_a,rule_b,dice,|A|,|B|
139,P-LBAL,P-TLI,0.63,42,85
137,P-HVI,RVT-IN,0.60,65,64
0,AAD-RD,AAD-RS,0.57,2,5
133,P-HSUMO,P-TLO,0.47,75,53
24,HASUMI,PGAV-IN,0.42,163,99
43,HNR-IN,RVT-IN,0.39,255,64
48,HNR-OUT,RVT-OUT,0.35,61,18
128,P-HSUMI,P-TLI,0.34,93,85
61,IN>%OUT,OUT>%IN,0.33,126,391
7,HANUMI,P-HVI,0.33,111,65


=== Subconjuntos por MISMA transacción (cliente, tx) ===


,subset_rule,superset_rule,|subset|,|superset|,flag
0,AAD-RD,AAD-RS,2,5,A ⊆ B


=== Similaridad (Dice) por MISMO DÍA (cliente, fecha) ===


,rule_a,rule_b,dice,|A|,|B|
0,AAD-RD,AAD-RS,0.67,1,2
177,P-LBAL,P-TLI,0.63,42,84
173,P-HVI,RVT-IN,0.57,33,44
167,P-HSUMO,P-TLO,0.51,62,52
32,HASUMI,PGAV-IN,0.47,133,98
157,P-HSUMI,P-TLI,0.38,70,84
64,HNR-OUT,RVT-OUT,0.37,49,16
58,HNR-IN,RVT-IN,0.37,184,44
79,IN>%OUT,OUT>%IN,0.32,74,211
181,P-LVAL,P-TLI,0.30,83,84


=== Subconjuntos por MISMO DÍA (cliente, fecha) ===


,subset_rule,superset_rule,|subset|,|superset|,flag
0,AAD-RD,AAD-RS,1,2,A ⊆ B


In [6]:
def always_with(df, key_cols, min_support=5):
    """
    Para cada regla A, mira si en (unidades) en que aparece A,
    B aparece en >= 95% de esos mismos casos (y viceversa opcional).
    """
    units = df[["rule_code"] + key_cols].drop_duplicates()
    # total unidades por regla
    tot = units.groupby("rule_code").size().to_dict()
    # mapa unidad -> set de reglas
    unit_to_rules = units.groupby(key_cols)["rule_code"].apply(set)

    # co-conteos A->B: cuántas veces que aparece A, también apareció B en esa unidad
    co = defaultdict(Counter)
    for _, rules in unit_to_rules.items():
        for a in rules:
            for b in (rules - {a}):
                co[a][b] += 1

    rows=[]
    for a, ctr in co.items():
        if tot.get(a,0) < min_support:
            continue
        for b, c in ctr.items():
            frac = c / tot[a]
            if frac >= 0.95:  # “siempre acompañada” (≈ 95% de los casos)
                rows.append((a,b, tot[a], c, frac))
    return pd.DataFrame(rows, columns=["rule_A","rule_B","support_A","co_hits","co_ratio_A→B"])

print("=== 'Siempre acompañada' por MISMA transacción ===")
display(always_with(with_tx, ["subject_id","tx_id"]))

print("=== 'Siempre acompañada' por MISMO DÍA ===")
display(always_with(expl, ["subject_id","created_date"]))


=== 'Siempre acompañada' por MISMA transacción ===


,rule_A,rule_B,support_A,co_hits,co_ratio_A→B
0,P-LBAL,P-TLI,42,40,0.95
1,RVT-IN,HNR-IN,64,62,0.97


=== 'Siempre acompañada' por MISMO DÍA ===


,rule_A,rule_B,support_A,co_hits,co_ratio_A→B
0,P-LBAL,P-TLI,42,40,0.95
1,RVT-IN,HNR-IN,44,42,0.95


In [7]:
# 0 alertas (en el CSV actual)
zero_alerts = by_rule[by_rule["total_alerts"]==0].copy()

# 0 sospechosos entre las revisadas
zero_tp = by_rule[(by_rule["reviewed"]>0) & (by_rule["suspicious"]==0)].copy()

# precisión muy baja (ej.: <1%) entre revisadas
low_precision = by_rule[(by_rule["reviewed"]>10) & (by_rule["precision%"]<1)].copy()

# pares con Dice alto (≥0.80) → posibles redundancias
redundant_tx  = pair_tx[pair_tx["dice"]>=0.80].copy()
redundant_day = pair_day[pair_day["dice"]>=0.80].copy()

print("=== Reglas con 0 alertas (CSV) ===")
display(zero_alerts)

print("=== Reglas con 0 sospechosos (entre revisadas) ===")
display(zero_tp)

print("=== Reglas con precisión < 1% (entre revisadas>10) ===")
display(low_precision)

print("=== Pares potencialmente redundantes (Dice ≥ 0.80) — MISMA transacción ===")
display(redundant_tx)

print("=== Pares potencialmente redundantes (Dice ≥ 0.80) — MISMO día ===")
display(redundant_day)

print("=== Subconjuntos claros (MISMA transacción) — un rule set contenido en otro ===")
display(subset_tx)

print("=== Subconjuntos claros (MISMO día) — un rule set contenido en otro ===")
display(subset_day)


=== Reglas con 0 alertas (CSV) ===


,rule_code,total_alerts,suspicious,not_suspicious,reviewed,fp_rate%,precision%


=== Reglas con 0 sospechosos (entre revisadas) ===


,rule_code,total_alerts,suspicious,not_suspicious,reviewed,fp_rate%,precision%
12,CDC02,62,0,62,62,100.00,0.00
24,SEC,19,0,19,19,100.00,0.00
26,RVT-OUT,16,0,16,16,100.00,0.00
31,VC,2,0,2,2,100.00,0.00
32,AAD-RS,2,0,1,1,100.00,0.00
33,AAD-LA,1,0,1,1,100.00,0.00


=== Reglas con precisión < 1% (entre revisadas>10) ===


...

,rule_a,rule_b,dice,|A|,|B|


=== Subconjuntos claros (MISMA transacción) — un rule set contenido en otro ===


,subset_rule,superset_rule,|subset|,|superset|,flag
0,AAD-RD,AAD-RS,2,5,A ⊆ B


=== Subconjuntos claros (MISMO día) — un rule set contenido en otro ===


,subset_rule,superset_rule,|subset|,|superset|,flag
0,AAD-RD,AAD-RS,1,2,A ⊆ B


In [8]:
SEG = "Investment Vehicle"  # cámbialo a otro sub-segmento si quieres

alerts_seg = alerts[alerts["customer_sub_type"].eq(SEG)].copy()
expl_seg   = alerts_seg.explode("tx_list").rename(columns={"tx_list":"tx_id"})
expl_seg["has_tx"] = expl_seg["tx_id"].notna()

# KPIs por regla (segmento)
by_rule_seg = (alerts_seg
   .groupby("rule_code")
   .agg(total_alerts=("alert_id","count"),
        suspicious=("status", lambda s: (s=="Suspicious").sum()),
        not_suspicious=("status", lambda s: (s=="Not Suspicious").sum()))
   .reset_index())
by_rule_seg["reviewed"]   = by_rule_seg["suspicious"] + by_rule_seg["not_suspicious"]
by_rule_seg["fp_rate%"]   = by_rule_seg.apply(lambda r: (r["not_suspicious"]/r["reviewed"]*100) if r["reviewed"] else 0, axis=1)
by_rule_seg["precision%"] = by_rule_seg.apply(lambda r: (r["suspicious"]/r["reviewed"]*100) if r["reviewed"] else 0, axis=1)
by_rule_seg = by_rule_seg.sort_values(["reviewed","total_alerts"], ascending=False).reset_index(drop=True)

print(f"=== Segmento: {SEG} — KPIs por regla ===")
display(by_rule_seg.head(30))

# Similaridad y subconjuntos dentro del segmento
def co_block(df, key_cols):
    if df.empty: 
        return (pd.DataFrame(columns=["rule_a","rule_b","dice","|A|","|B|"]),
                pd.DataFrame(columns=["subset_rule","superset_rule","|subset|","|superset|","flag"]))
    units_by_rule = df.groupby("rule_code")[key_cols].apply(
        lambda x: set(map(tuple, x.drop_duplicates().to_records(index=False)))
    ).to_dict()
    rules = sorted(units_by_rule.keys())
    rows=[]
    for a,b in combinations(rules,2):
        A,B = units_by_rule[a], units_by_rule[b]
        if len(A)==0 or len(B)==0: 
            continue
        s = 2*len(A & B)/(len(A)+len(B))
        if s>0:
            rows.append((a,b,s,len(A),len(B)))
    pair = pd.DataFrame(rows, columns=["rule_a","rule_b","dice","|A|","|B|"]).sort_values("dice", ascending=False)

    subset_rows=[]
    for a,b in combinations(rules,2):
        A,B = units_by_rule[a], units_by_rule[b]
        if A and A.issubset(B): subset_rows.append((a,b,len(A),len(B),"A ⊆ B"))
        if B and B.issubset(A): subset_rows.append((b,a,len(B),len(A),"B ⊆ A"))
    subs = pd.DataFrame(subset_rows, columns=["subset_rule","superset_rule","|subset|","|superset|","flag"]).drop_duplicates()
    return pair, subs

print("\n=== Segmento: Co-similaridad (MISMA transacción) ===")
pair_tx_seg, subset_tx_seg = co_block(expl_seg[expl_seg["has_tx"]][["rule_code","subject_id","tx_id"]], ["subject_id","tx_id"])
display(pair_tx_seg.head(30)); display(subset_tx_seg.head(30))

print("\n=== Segmento: Co-similaridad (MISMO día) ===")
pair_day_seg, subset_day_seg = co_block(expl_seg[["rule_code","subject_id","created_date"]], ["subject_id","created_date"])
display(pair_day_seg.head(30)); display(subset_day_seg.head(30))


=== Segmento: Investment Vehicle — KPIs por regla ===


,rule_code,total_alerts,suspicious,not_suspicious,reviewed,fp_rate%,precision%
0,P-LVAL,31,3,28,31,90.32,9.68
1,OUT>%IN,22,1,21,22,95.45,4.55
2,PGAV-OUT,18,0,18,18,100.00,0.00
3,NCU,14,1,13,14,92.86,7.14
4,HANUMI,11,1,10,11,90.91,9.09
5,HASUMI,10,0,10,10,100.00,0.00
6,HANUMO,9,1,8,9,88.89,11.11
7,HASUMO,9,0,9,9,100.00,0.00
8,P-HSUMO,8,0,8,8,100.00,0.00
9,P-TLO,8,0,8,8,100.00,0.00



=== Segmento: Co-similaridad (MISMA transacción) ===


,rule_a,rule_b,dice,|A|,|B|
29,P-LBAL,P-TLI,1.00,4,4
34,P-TLO,PGAV-OUT,0.54,8,18
27,P-HSUMO,P-TLO,0.50,8,8
30,P-LBAL,PGAV-IN,0.40,4,1
33,P-TLI,PGAV-IN,0.40,4,1
9,IN-OUT-1,OUT>AVG,0.40,3,7
6,HASUMI,P-TLI,0.38,12,4
4,HASUMI,P-LBAL,0.38,12,4
28,P-HSUMO,PGAV-OUT,0.31,8,18
17,IN>%OUT,P-TLO,0.31,5,8


,subset_rule,superset_rule,|subset|,|superset|,flag
0,P-LBAL,P-TLI,4,4,A ⊆ B
1,P-TLI,P-LBAL,4,4,B ⊆ A
2,PGAV-IN,P-LBAL,1,4,B ⊆ A
3,PGAV-IN,P-TLI,1,4,B ⊆ A



=== Segmento: Co-similaridad (MISMO día) ===


,rule_a,rule_b,dice,|A|,|B|
30,P-LBAL,P-TLI,1.00,4,4
35,P-TLO,PGAV-OUT,0.56,8,17
28,P-HSUMO,P-TLO,0.50,8,8
4,HASUMI,P-LBAL,0.46,9,4
6,HASUMI,P-TLI,0.46,9,4
10,IN-OUT-1,OUT>AVG,0.44,3,6
31,P-LBAL,PGAV-IN,0.40,4,1
34,P-TLI,PGAV-IN,0.40,4,1
18,IN>%OUT,P-TLO,0.33,4,8
29,P-HSUMO,PGAV-OUT,0.32,8,17


,subset_rule,superset_rule,|subset|,|superset|,flag
0,P-LBAL,P-TLI,4,4,A ⊆ B
1,P-TLI,P-LBAL,4,4,B ⊆ A
2,PGAV-IN,P-LBAL,1,4,B ⊆ A
3,PGAV-IN,P-TLI,1,4,B ⊆ A
